In [5]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv('CensusBase_SDGV1.csv')

In [24]:
df.head(5)

,Building,lat,lon,SchoolScore,HospitalScore,PoliceScore,WasteManagementScore,ElderlyCareScore,ChildCareScore,TotalScore,...,Region,District,TotalPopulation,length,Income,1-3,4,5+,Sex ratio,"Import/export, wholesale and retail trades, accommodation and food services"
0,Providence Peak,22.432130,114.202489,0.05,0.01,0.09,0.0,0.0,0.0,0.05,...,NewTerritories,TaiPo,2115.33,3,61320.0,46.0%,23.1%,31.0%,760,28.3%
1,(Sha Tau Kok) residential buildings,22.531577,114.203240,0.01,0.04,0.12,0.0,0.0,0.0,0.06,...,NewTerritories,North,1523.50,2,16230.0,44.5%,28.9%,26.6%,942,26.1%
2,"(Yuen Long Southeast) Tower 1-5, The Reach (Sh...",22.437429,114.027660,0.27,0.17,0.00,0.0,0.0,0.0,0.17,...,NewTerritories,YuenLong,1841.00,1,34000.0,89.8%,10.2%,0.0%,698,31.4%
3,"(Yuen Long Southeast) Tower 10-13, The Reach (...",22.437429,114.027660,0.24,0.15,0.00,0.0,0.0,0.0,0.15,...,NewTerritories,YuenLong,2076.00,1,38060.0,77.4%,17.5%,5.1%,829,20.9%
4,(Yuen Long Southeast) Grand Del Sol (Fung Cheu...,22.440560,114.032987,0.11,0.08,0.00,0.0,0.0,0.0,0.07,...,NewTerritories,YuenLong,3696.00,1,50850.0,51.2%,29.3%,19.6%,815,23.3%


In [8]:
for i in df.index:
    df['Building'][i] = df['Building'].iloc[i].strip()

In [9]:
#df.columns

In [10]:
sdg1 = df.copy()

In [11]:
sdg1 = sdg1[['District/Area/Building Group', 'Building', 'lat', 'lon', 'Region', 'District', 'TotalPopulation', 'Income',
            '1-3', '4', '5+', 'Sex ratio',]]

In [12]:
sdg1['1.1.1'] = None
sdg1['1.2.1'] = None
sdg1['1.4.1'] = None
sdg1['1.a.2'] = None

In [13]:
#sdg1['Sex ratio'].replace({',':''}, inplace = True)

In [14]:
for i in sdg1.index:
    inc = sdg1['Income'].iloc[i]
    pop = sdg1['TotalPopulation'].sum()
    if inc<=500:
        a = (sdg1['TotalPopulation'].iloc[i]/pop)*100
        sdg1['1.1.1'].iloc[i] = 0 - a
    else:
        sdg1['1.1.1'].iloc[i] = 0

In [15]:
for i in sdg1.index:
    sdg1['1-3'].iloc[i] = sdg1['1-3'].iloc[i][: -1]
    sdg1['4'].iloc[i] = sdg1['4'].iloc[i][: -1]
    sdg1['5+'].iloc[i] = sdg1['5+'].iloc[i][: -1]

In [16]:
sdg1['Sex ratio']=sdg1['Sex ratio'].str.replace(',','')

In [17]:
for i in sdg1.columns:
    try:
        sdg1[i] = sdg1[i].astype(float)
    except:
        print('Error')

Error
Error
Error
Error


In [18]:
for i in sdg1.index:
    inc = sdg1['Income'].iloc[i]
    val = 0
    if inc <= 10000:
        a = sdg1['TotalPopulation'].iloc[i] * sdg1['1-3'].iloc[i]
        a = a/sdg1['TotalPopulation'].sum()
        val = val+a
        if inc <= 20800:
            a = sdg1['TotalPopulation'].iloc[i] * sdg1['4'].iloc[i]
            a = a/sdg1['TotalPopulation'].sum()
            val = val+a
            if inc <= 21000:
                a = sdg1['TotalPopulation'].iloc[i] * sdg1['5+'].iloc[i]
                a = a/sdg1['TotalPopulation'].sum()
                val = val+a
    sdg1['1.2.1'].iloc[i] = 0-val

In [19]:
sdg1['Male'] = (sdg1['Sex ratio'] * sdg1['TotalPopulation'])/1000
sdg1['Female'] = sdg1['TotalPopulation'] - sdg1['Male']
sdg1['Male'] = sdg1['Male'].round(2)
sdg1['Female'] = sdg1['Female'].round(2)

In [20]:
sdg1.drop(columns= ['Sex ratio'], inplace = True)

In [25]:
sdg1['1.a.2'] = df['SchoolScore'] + df['HospitalScore']

In [26]:
sdg1['1.4.1'] = df['TotalScore']

In [27]:
sdg1 = sdg1[['District/Area/Building Group', 'Building', 'lat', 'lon', 'Region',
       'District', 'TotalPopulation', 'Male', 'Female', 'Income', '1-3', '4', '5+', '1.1.1',
       '1.2.1', '1.4.1', '1.a.2']]

In [28]:
sdg1.shape

(6983, 17)

In [29]:
sdg1['SDG1'] = sdg1['1.1.1'] + sdg1['1.2.1'] + sdg1['1.4.1'] + sdg1['1.a.2']

In [30]:
sdg1 = sdg1[['Building', 'Male', 'Female', '1-3', '4', '5+', '1.1.1', '1.2.1', '1.4.1', '1.a.2', 'SDG1']]

In [31]:
df = pd.merge(df, sdg1, on = 'Building', how='inner')

In [ ]:
df.drop_duplicates(subset = ['Building', 'lat', 'lon'], inplace = True)

In [ ]:
df = df.reset_index()
df.drop(columns=['index'], inplace = True)

In [ ]:
df

In [ ]:
sdg2 = df.copy()

In [ ]:
sdg2 = sdg2[['Building', 'District/Area/Building Group', 'lat', 'lon', 'Region',
       'District', 'TotalPopulation', 'length', 'Income', 'Address']]

In [ ]:
sdg2['2.3.2'] = None
sdg2['SDG2'] = None

In [ ]:
df2 = pd.read_csv('CentamapDistrictRegion.csv')

In [ ]:
df2 = df2[['District/Area/Building Group', 'Import/export, wholesale and retail trades, accommodation and food services']]

In [ ]:
sdg2 = pd.merge(df2, sdg2, on = 'District/Area/Building Group', how='inner')

In [ ]:
for i in sdg2.index:
    sdg2['Import/export, wholesale and retail trades, accommodation and food services'].iloc[i] = sdg2['Import/export, wholesale and retail trades, accommodation and food services'].iloc[i][: -1]

In [ ]:
for i in sdg2.columns:
    try:
        sdg2[i] = sdg2[i].astype(float)
    except:
        print('Error')

In [ ]:
for i in sdg2.index:
    inc = sdg2['Income'].iloc[i]
    food = sdg2['Import/export, wholesale and retail trades, accommodation and food services'].iloc[i]
    food = food/100
    pop = sdg2['TotalPopulation'].iloc[i]
    foodpop = food*pop
    a = (inc * foodpop)/pop
    sdg2['2.3.2'].iloc[i] = a
    sdg2['SDG2'].iloc[i] = a

In [ ]:
sdg2 = sdg2[['Building', '2.3.2', 'SDG2']]

In [ ]:
df = pd.merge(df, sdg2, left_index=True, right_index=True, how='inner')

In [ ]:
df

In [ ]:
df.rename(columns = {'Building_x' : 'Building'}, inplace = True)
df.drop(columns = ['Building_y'], inplace = True)